## Fine tuning Chat GPT 2.0

In [ ]:
!pip install transformers datasets

### Install Required Libraries

### Import Libraries

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

### Load the Pre-trained Model and Tokenizer

In [ ]:
# Load the pre-trained model and tokenizer
model_name = 'gpt2'  # You can use 'gpt2-medium', 'gpt2-large', or 'gpt2-xl' for larger models
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move the model to the GPU if available
if torch.cuda.is_available():
    model = model.to('cuda')

### Prepare Your Dataset

In [ ]:
# Load a text dataset from a local file (replace 'path/to/your/dataset.txt' with your dataset path)
# The text file should contain your training examples separated by new lines

dataset = load_dataset('text', data_files='path/to/your/dataset.txt')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

### Set Up Training Arguments

In [11]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory for the model predictions and checkpoints
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=2,   # batch size for training
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
)


NameError: name 'TrainingArguments' is not defined

### Create the Trainer


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_dataset['train'],  # training dataset
    eval_dataset=tokenized_dataset['train']  # evaluation dataset (using the same for simplicity)
)


### Train the Model

In [ ]:
trainer.train()

### Save the Model

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')


###  Using the Fine-tuned Model

In [ ]:
# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

# Generate text using the fine-tuned model
input_text = "Natural Language Processing"
input_ids = fine_tuned_tokenizer.encode(input_text, return_tensors='pt')

# Move to GPU if available
if torch.cuda.is_available():
    input_ids = input_ids.to('cuda')

output = fine_tuned_model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)